### 히스토그램 계산을 통한 관련 이미지 검색

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 주소 기본값 및 저장 리스트
fath = 'c:/data/temp/'
jpg = '.jpg'
name = ['bottle', 'mouse']
source = [['' for i in range(10)] for j in range(10)]

result = [['' for i in range(10)] for j in range(10)]

# 이미지 주소 추가
for i in range(1, 7):
    source[0][i-1] = (fath+name[0]+f'{i}'+jpg)
for i in range(1, 6):
    source[1][i-1] = (fath+name[1]+f'{i}'+jpg)

# 이미지 주소 출력
for i in range(2):
    print(source[i])

# 히스토그램 계산
def calculate_histogram(image, i, j):
    global result
    # 이미지 불러오기
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100))
    
    # 히스토그램 계산
    hist = cv2.calcHist([img], [0], None, [64], [0, 256])
    result[i][j] = hist
    # Plot histogram
    # plt.figure()
    # plt.title('Histogram of ' + image)
    # plt.xlabel('Pixel Value')
    # plt.ylabel('Frequency')
    # plt.plot(hist)
    # plt.show()

# Iterate over the image filenames
for i in range(len(source)):
    for j in range(len(source[i])):
        if source[i][j] == '':
            break
        calculate_histogram(source[i][j], i, j)


['c:/data/temp/bottle1.jpg', 'c:/data/temp/bottle2.jpg', 'c:/data/temp/bottle3.jpg', 'c:/data/temp/bottle4.jpg', 'c:/data/temp/bottle5.jpg', 'c:/data/temp/bottle6.jpg', '', '', '', '']
['c:/data/temp/mouse1.jpg', 'c:/data/temp/mouse2.jpg', 'c:/data/temp/mouse3.jpg', 'c:/data/temp/mouse4.jpg', 'c:/data/temp/mouse5.jpg', '', '', '', '', '']


### VLAD 코드 예시

In [1]:
import numpy as np

def vlad_encoding(hist_results):
    # Check if there are any histograms
    if len(hist_results) == 0:
        return None

    # Get the dimensions of the histograms
    hist_dim = len(hist_results[0])

    # Initialize the VLAD vector
    vlad_vector = np.zeros(hist_dim)

    # Compute the mean vector
    mean_vector = np.mean(hist_results, axis=0)

    # Iterate over each histogram result
    for hist in hist_results:
        # Compute the residuals
        residuals = hist - mean_vector

        # Accumulate the residuals to the VLAD vector
        vlad_vector += residuals

    # L2 normalize the VLAD vector
    vlad_vector /= np.linalg.norm(vlad_vector)

    return vlad_vector

# Example usage
histogram_results = [
    [0.2, 0.1, 0.3, 0.4],
    [0.1, 0.4, 0.2, 0.3],
    [0.3, 0.2, 0.4, 0.1]
]

vlad_vector = vlad_encoding(histogram_results)
print(vlad_vector)

[-0.65759595  0.65759595  0.32879797  0.16439899]


### 여러장의 hist 에서 VLAD 구하기 예시

In [ ]:
import cv2
import numpy as np

def calculate_histogram(image):
    # Load the image
    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    
    # Calculate histogram
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    
    return hist.flatten()

# List of image filenames
image_filenames = ['image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg',
                   'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg']

# List to store the histograms
histograms = []

# Iterate over the image filenames
for image_filename in image_filenames:
    histogram = calculate_histogram(image_filename)
    histograms.append(histogram)

# Convert the histograms list to a numpy array
histograms = np.array(histograms)

# Compute the mean vector (VLAD encoding)
mean_vector = np.mean(histograms, axis=0)
vlad_vector = mean_vector / np.linalg.norm(mean_vector)

print(vlad_vector)

### faiss 예제

* pip install faiss-cpu
* 페이스북에서 만든 건 업데이트를 안해서 3.7 이하에서만 가능

* 설명 참고
* https://euriion.com/?p=413162
* https://velog.io/@gtpgg1013/Faiss-%EC%9C%A0%EC%82%AC%EB%8F%84-%EA%B3%84%EC%82%B0%EC%9D%84-%EB%B9%A0%EB%A5%B4%EA%B3%A0-%EA%B0%84%ED%8E%B8%ED%9E%88
* 

In [ ]:
import numpy as np
import faiss

# Generate some random vectors
d = 64  # 벡터의 차원
n = 1000  # 벡터의 수

# 임의의 데이터 세트 생성
np.random.seed(0)
dataset = np.random.random((n, d)).astype('float32')

#  거리 측정값이 있는 'faiss.IndexFlatL2'를 사용하여 인덱스를 생성
index = faiss.IndexFlatL2(d)  # L2 distance measure

# 데이터셋을 인덱스에 추가
index.add(dataset)

# 유사성 검색을 수행하기 위해 검색할 가장 가까운 이웃의 수(k)를 지정하고 모양 (1, d)의 임의 쿼리 벡터 생성
k = 5  # 가까운 이웃의 수
query = np.random.random((1, d)).astype('float32')  # Random query vector

# 쿼리 벡터에 대해 k 가장 가까운 이웃을 검색하기 위해 index.search를 사용합
# 결과 거리와 인덱스는 각각 'distances'와 'indices'에 저장
distances, indices = index.search(query, k)

# 쿼리 벡터
print('Query vector(쿼리 벡터):')
print(query)

# 가장 가까운 k개의 이웃
print('Nearest neighbors(가까운 이웃):')
print(dataset[indices[0]])

# 거리
print('Distances(거리):')
print(distances[0])
